# End to end examples

## Horse colic data

In [1]:
using Pkg; Pkg.activate("D:/JULIA/6_ML_with_Julia/EX-horse"); Pkg.instantiate()

  Activating project at `D:\JULIA\6_ML_with_Julia\EX-horse`


> Initial data processing
> 1. Getting the data
> 2. Inspecting columns
> 3. Dealing with missing values

> A baseline model <br>
> Trying another model

### Initial data processing
---

In this example, we consider the UCI "horse colic" dataset

This is a reasonably messy classification problem with missing values etc and so some work should be expected in the feature processing.

### Getting the data


The data is pre-split in training and testing and we will keep it as such

In [2]:
using MLJ
using HTTP
using CSV
import DataFrames: DataFrame, select!, Not
req1 = HTTP.get("http://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data")
req2 = HTTP.get("http://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.test")
header = ["surgery", "age", "hospital_number",
    "rectal_temperature", "pulse",
    "respiratory_rate", "temperature_extremities",
    "peripheral_pulse", "mucous_membranes",
    "capillary_refill_time", "pain",
    "peristalsis", "abdominal_distension",
    "nasogastric_tube", "nasogastric_reflux",
    "nasogastric_reflux_ph", "feces", "abdomen",
    "packed_cell_volume", "total_protein",
    "abdomcentesis_appearance", "abdomcentesis_total_protein",
    "outcome", "surgical_lesion", "lesion_1", "lesion_2", "lesion_3",
    "cp_data"]

28-element Vector{String}:
 "surgery"
 "age"
 "hospital_number"
 "rectal_temperature"
 "pulse"
 "respiratory_rate"
 "temperature_extremities"
 "peripheral_pulse"
 "mucous_membranes"
 "capillary_refill_time"
 "pain"
 "peristalsis"
 "abdominal_distension"
 ⋮
 "feces"
 "abdomen"
 "packed_cell_volume"
 "total_protein"
 "abdomcentesis_appearance"
 "abdomcentesis_total_protein"
 "outcome"
 "surgical_lesion"
 "lesion_1"
 "lesion_2"
 "lesion_3"
 "cp_data"

In [3]:
csv_opts = (header = header, delim = ' ', missingstring = "?", ignorerepeated = true)

data_train = CSV.read(req1.body, DataFrame; csv_opts...)
data_test = CSV.read(req2.body, DataFrame; csv_opts...)
@show size(data_train)
@show size(data_test)

size(data_train) = (300, 28)
size(data_test) = (68, 28)


(68, 28)

### Inspecting columns

To simplify the analysis, we will drop the columns `Lesion *` as they would need specific re-encoding which would distract us a bit.

In [4]:
data_train[1:12, :lesion_1]

12-element Vector{Int64}:
 11300
  2208
     0
  2208
  4300
     0
  3124
  2208
  3205
     0
  2124
  2111

In [5]:
data_train[1:12, :lesion_2]

12-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [6]:
data_train[1:12, :lesion_3]

12-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [7]:
unwanted = [:lesion_1, :lesion_2, :lesion_3]
data = vcat(data_train, data_test)
select!(data, Not(unwanted));

Let's also keep track of the initial train-test split

In [8]:
train = 1:nrows(data_train)
test = last(train) .+ (1:nrows(data_test));

In [9]:
train

1:300

In [10]:
test

301:368

We know from reading the description that some of these features represent multiclass data; to facilitate the interpretation, we can use `autotype` from `ScientificTypes`. By default, `autotype` will check all columns and suggest a Finite type assuming there are relatively few distinct values in the column. More sophisticated rules can be passed, see ScientificTypes.jl:

In [11]:
datac = coerce(data, autotype(data));

Let's see column by column whether it looks ok now

In [12]:
sch = schema(datac)

┌─────────────────────────────┬───────────────────────────────────┬─────────────
│ names                       │ scitypes                          │ types      ⋯
├─────────────────────────────┼───────────────────────────────────┼─────────────
│ surgery                     │ Union{Missing, OrderedFactor{2}}  │ Union{Miss ⋯
│ age                         │ OrderedFactor{2}                  │ Categorica ⋯
│ hospital_number             │ Count                             │ Int64      ⋯
│ rectal_temperature          │ Union{Missing, Continuous}        │ Union{Miss ⋯
│ pulse                       │ Union{Missing, Count}             │ Union{Miss ⋯
│ respiratory_rate            │ Union{Missing, Count}             │ Union{Miss ⋯
│ temperature_extremities     │ Union{Missing, OrderedFactor{4}}  │ Union{Miss ⋯
│ peripheral_pulse            │ Union{Missing, OrderedFactor{4}}  │ Union{Miss ⋯
│ mucous_membranes            │ Union{Missing, OrderedFactor{6}}  │ Union{Miss ⋯
│ capillary_refill_time     

In [13]:
for (name, scitype) in zip(sch.names, sch.scitypes)
    println(rpad("$name", 30), scitype)
end

surgery                       Union{Missing, OrderedFactor{2}}
age                           OrderedFactor{2}
hospital_number               Count
rectal_temperature            Union{Missing, Continuous}
pulse                         Union{Missing, Count}
respiratory_rate              Union{Missing, Count}
temperature_extremities       Union{Missing, OrderedFactor{4}}
peripheral_pulse              Union{Missing, OrderedFactor{4}}
mucous_membranes              Union{Missing, OrderedFactor{6}}
capillary_refill_time         Union{Missing, OrderedFactor{3}}
pain                          Union{Missing, OrderedFactor{5}}
peristalsis                   Union{Missing, OrderedFactor{4}}
abdominal_distension          Union{Missing, OrderedFactor{4}}
nasogastric_tube              Union{Missing, OrderedFactor{3}}
nasogastric_reflux            Union{Missing, OrderedFactor{3}}
nasogastric_reflux_ph         Union{Missing, OrderedFactor{24}}
feces                         Union{Missing, OrderedFactor{4}}

Most columns are now treated as either Multiclass or Ordered, this corresponds to the description of the data. For instance:

* `Surgery` is described as `1=yes / 2=no`

* `Age` is described as `1=adult / 2=young`

Inspecting the rest of the descriptions and the current scientific type, there are a few more things that can be observed:

* hospital number is still a count, this means that there are relatively many hospitals and so that's probably not very useful,

* pulse and respiratory rate are still as count but the data description suggests that they can be considered as continuous

> Data Set Information:
> 
> 2 data files:
> -- horse-colic.data: 300 training instances
> -- horse-colic.test: 68 test instances
> 
> Possible class attributes: 24 (whether lesion is surgical)
> -- others include: 23, 25, 26, and 27
> 
> Many Data types: (continuous, discrete, and nominal)
> 
> 
> Attribute Information:
> 
> 1: surgery?
> 1 = Yes, it had surgery
> 2 = It was treated without surgery
> 
> 2: Age
> 1 = Adult horse
> 2 = Young (< 6 months)
> 
> 3: Hospital Number
> - numeric id
> - the case number assigned to the horse (may not be unique if the horse is treated > 1 time)
> 
> 4: rectal temperature
> - linear
> - in degrees celsius.
> - An elevated temp may occur due to infection.
> - temperature may be reduced when the animal is in late shock
> - normal temp is 37.8
> - this parameter will usually change as the problem progresses, eg. may start out normal, then become elevated because of the lesion, passing back through the normal range as the horse goes into shock
> 5: pulse
> - linear
> - the heart rate in beats per minute
> - is a reflection of the heart condition: 30 -40 is normal for adults
> - rare to have a lower than normal rate although athletic horses may have a rate of 20-25
> - animals with painful lesions or suffering from circulatory shock may have an elevated heart rate
> 
> 6: respiratory rate
> - linear
> - normal rate is 8 to 10
> - usefulness is doubtful due to the great fluctuations
> 
> 7: temperature of extremities
> - a subjective indication of peripheral circulation
> - possible values:
> 1 = Normal
> 2 = Warm
> 3 = Cool
> 4 = Cold
> - cool to cold extremities indicate possible shock
> - hot extremities should correlate with an elevated rectal temp.
> 
> 8: peripheral pulse
> - subjective
> - possible values are:
> 1 = normal
> 2 = increased
> 3 = reduced
> 4 = absent
> - normal or increased p.p. are indicative of adequate circulation while reduced or absent indicate poor perfusion
> 
> 9: mucous membranes
> - a subjective measurement of colour
> - possible values are:
> 1 = normal pink
> 2 = bright pink
> 3 = pale pink
> 4 = pale cyanotic
> 5 = bright red / injected
> 6 = dark cyanotic
> - 1 and 2 probably indicate a normal or slightly increased circulation
> - 3 may occur in early shock
> - 4 and 6 are indicative of serious circulatory compromise
> - 5 is more indicative of a septicemia
> 
> 10: capillary refill time
> - a clinical judgement. The longer the refill, the poorer the circulation
> - possible values
> 1 = < 3 seconds
> 2 = >= 3 seconds
> 
> 11: pain - a subjective judgement of the horse's pain level
> - possible values:
> 1 = alert, no pain
> 2 = depressed
> 3 = intermittent mild pain
> 4 = intermittent severe pain
> 5 = continuous severe pain
> - should NOT be treated as a ordered or discrete variable!
> - In general, the more painful, the more likely it is to require surgery
> - prior treatment of pain may mask the pain level to some extent
> 
> 12: peristalsis
> - an indication of the activity in the horse's gut. As the gut becomes more distended or the horse becomes more toxic, the activity decreases
> - possible values:
> 1 = hypermotile
> 2 = normal
> 3 = hypomotile
> 4 = absent
> 
> 13: abdominal distension
> - An IMPORTANT parameter.
> - possible values
> 1 = none
> 2 = slight
> 3 = moderate
> 4 = severe
> - an animal with abdominal distension is likely to be painful and have reduced gut motility.
> - a horse with severe abdominal distension is likely to require surgery just tio relieve the pressure
> 
> 14: nasogastric tube
> - this refers to any gas coming out of the tube
> - possible values:
> 1 = none
> 2 = slight
> 3 = significant
> - a large gas cap in the stomach is likely to give the horse discomfort
> 
> 15: nasogastric reflux
> - possible values
> 1 = none
> 2 = > 1 liter
> 3 = < 1 liter
> - the greater amount of reflux, the more likelihood that there is some serious obstruction to the fluid passage from the rest of the intestine
> 
> 16: nasogastric reflux PH
> - linear
> - scale is from 0 to 14 with 7 being neutral
> - normal values are in the 3 to 4 range
> 
> 17: rectal examination - feces
> - possible values
> 1 = normal
> 2 = increased
> 3 = decreased
> 4 = absent
> - absent feces probably indicates an obstruction
> 
> 18: abdomen
> - possible values
> 1 = normal
> 2 = other
> 3 = firm feces in the large intestine
> 4 = distended small intestine
> 5 = distended large intestine
> - 3 is probably an obstruction caused by a mechanical impaction and is normally treated medically
> - 4 and 5 indicate a surgical lesion
> 
> 19: packed cell volume
> - linear
> - the # of red cells by volume in the blood
> - normal range is 30 to 50. The level rises as the circulation becomes compromised or as the animal becomes dehydrated.
> 
> 20: total protein
> - linear
> - normal values lie in the 6-7.5 (gms/dL) range
> - the higher the value the greater the dehydration
> 
> 21: abdominocentesis appearance
> - a needle is put in the horse's abdomen and fluid is obtained from
> the abdominal cavity
> - possible values:
> 1 = clear
> 2 = cloudy
> 3 = serosanguinous
> - normal fluid is clear while cloudy or serosanguinous indicates a compromised gut
> 
> 22: abdomcentesis total protein
> - linear
> - the higher the level of protein the more likely it is to have a compromised gut. Values are in gms/dL
> 
> 23: outcome
> - what eventually happened to the horse?
> - possible values:
> 1 = lived
> 2 = died
> 3 = was euthanized
> 
> 24: surgical lesion?
> - retrospectively, was the problem (lesion) surgical?
> - all cases are either operated upon or autopsied so that this value and the lesion type are always known
> - possible values:
> 1 = Yes
> 2 = No
> 
> 25, 26, 27: type of lesion
> - first number is site of lesion
> 1 = gastric
> 2 = sm intestine
> 3 = lg colon
> 4 = lg colon and cecum
> 5 = cecum
> 6 = transverse colon
> 7 = retum/descending colon
> 8 = uterus
> 9 = bladder
> 11 = all intestinal sites
> 00 = none
> - second number is type
> 1 = simple
> 2 = strangulation
> 3 = inflammation
> 4 = other
> - third number is subtype
> 1 = mechanical
> 2 = paralytic
> 0 = n/a
> - fourth number is specific code
> 1 = obturation
> 2 = intrinsic
> 3 = extrinsic
> 4 = adynamic
> 5 = volvulus/torsion
> 6 = intussuption
> 7 = thromboembolic
> 8 = hernia
> 9 = lipoma/slenic incarceration
> 10 = displacement
> 0 = n/a
> 28: cp_data
> - is pathology data present for this case?
> 1 = Yes
> 2 = No
> - this variable is of no significance since pathology data is not included or collected for these cases
> 
> 

In [14]:
length(unique(datac.hospital_number))

346

yeah let's drop that

In [15]:
datac = select!(datac, Not(:hospital_number));

let's also coerce the pulse and respiratory rate, in fact we can do that with `autotype` specifying as rule the `discrete_to_continuous`

In [16]:
datac = coerce(datac, autotype(datac, rules = (:discrete_to_continuous, )));

### Dealing with missing values

There's quite a lot of missing values, in this tutorial we'll be a bit rough in how we deal with them applying the following rules of thumb:

* drop the rows where the outcome is unknown

* drop columns with more than 20% missing values

* simple imputation of whatever's left

In [17]:
missing_outcome = ismissing.(datac.outcome)
idx_missing_outcome = missing_outcome |> findall

2-element Vector{Int64}:
 133
 309

Ok there's only two row which is nice, let's remove them from the train/test indices and drop the rows

In [18]:
# train = setdiff!(train |> collect, idx_missing_outcome)
# test = setdiff!(test |> collect, idx_missing_outcome)
datac = datac[.!missing_outcome, :];

In [19]:
datac |> last

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_extremities
,Cat…?,Cat…,Float64?,Float64?,Float64?,Cat…?
366,2,1,37.6,88.0,36.0,3


Now let's look at how many missings there are per features

In [20]:
for name in names(datac)
    col = datac[:, name]
    ratio_missing = sum(ismissing.(col)) / nrows(datac) *100
    println(rpad(name, 30), round(ratio_missing, sigdigits = 3))
end

surgery                       0.0
age                           0.0
rectal_temperature            18.9
pulse                         7.1
respiratory_rate              19.4
temperature_extremities       17.5
peripheral_pulse              22.7
mucous_membranes              13.1
capillary_refill_time         10.4
pain                          17.2
peristalsis                   13.9
abdominal_distension          17.5
nasogastric_tube              35.5
nasogastric_reflux            36.1
nasogastric_reflux_ph         81.1
feces                         34.7
abdomen                       39.1
packed_cell_volume            9.84
total_protein                 11.5
abdomcentesis_appearance      52.7
abdomcentesis_total_protein   63.9
outcome                       0.0
surgical_lesion               0.0
cp_data                       0.0


Let's drop the ones with more than 20% (quite a few!)

In [21]:
unwanted = [:peripheral_pulse, :nasogastric_tube, :nasogastric_reflux,
        :nasogastric_reflux_ph, :feces, :abdomen, :abdomcentesis_appearance, :abdomcentesis_total_protein]
select!(datac, Not(unwanted));

Note that we could have done this better and investigated the nature of the features for which there's a lot of missing values but don't forget that our goal is to showcase MLJ!

Let's conclude by filling all missing values and separating the feature matrix from the target

In [22]:
@load FillImputer
filler = machine(FillImputer(), datac)
fit!(filler)
datac = MLJ.transform(filler, datac)

y, X = unpack(datac, ==(:outcome), name -> true);
X = coerce(X, autotype(X, :discrete_to_continuous));

import MLJModels

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\jeffr\.julia\packages\MLJModels\tMgLW\src\loading.jl:168


 ✔


┌ Info: Training Machine{FillImputer,…}.
└ @ MLJBase C:\Users\jeffr\.julia\packages\MLJBase\MuLnJ\src\machines.jl:464


### A baseline model

---

Let's define a first sensible model and get a baseline, basic steps are:

* one-hot-encode the categoricals

* feed all this into a classifier

In [23]:
@load OneHotEncoder
MultinomialClassifier = @load MultinomialClassifier pkg = "MLJLinearModels"

import MLJModels ✔
import MLJLinearModels

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\jeffr\.julia\packages\MLJModels\tMgLW\src\loading.jl:168
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\jeffr\.julia\packages\MLJModels\tMgLW\src\loading.jl:168


 ✔


MLJLinearModels.MultinomialClassifier

Let's have convenient handles over the training data

In [24]:
Xtrain = X[1:299, :]
ytrain = y[1:299];

Xtest =  X[300:366, :]
ytest =  X[300:366, :]

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_extremities
,Cat…,Cat…,Float64,Float64,Float64,Cat…
1,2,1,38.5,54.0,20.0,3
2,2,1,37.6,48.0,36.0,3
3,1,1,37.7,44.0,28.0,3
4,1,1,37.0,56.0,24.0,3
5,2,1,38.0,42.0,12.0,3
6,1,1,38.1,60.0,40.0,3
7,2,1,38.4,80.0,60.0,3
8,2,1,37.8,48.0,12.0,2
9,2,1,37.9,45.0,36.0,3


And let's define a pipeline corresponding to the operations above

In [25]:
SimplePipe = Pipeline(
        OneHotEncoder(), 
        MultinomialClassifier(),
        prediction_type=:probabilistic
)

mach = machine(SimplePipe, Xtrain, ytrain)

res = evaluate!(
    mach;
    resampling = Holdout(fraction_train = 0.9),
    measure=cross_entropy
)

round(res.measurement[1], sigdigits = 3)

0.859

This is the cross entropy on some held-out 10% of the training set. We can also just for the sake of getting a baseline, see the misclassification on the whole training data:

In [26]:
mcr = misclassification_rate(predict_mode(mach, Xtrain), ytrain)
println(rpad("MNC mcr : ", 10), round(mcr, sigdigits = 3))

MNC mcr : 0.298


That's not bad at all actually. Let's tune it a bit and see if we can get a bit better than that, not much point in going crazy, we might get a few percents but not much more.

In [27]:
model = SimplePipe
lambdas = range(model, :(multinomial_classifier.lambda), lower = 1e-3, upper = 100, scale = :log10)
tm = TunedModel(model=SimplePipe, ranges = lambdas, measure = cross_entropy)
mtm = machine(tm, Xtrain, ytrain)
fit!(mtm)
best_pipe = fitted_params(mtm).best_model

┌ Info: Training Machine{ProbabilisticTunedModel{Grid,…},…}.
└ @ MLJBase C:\Users\jeffr\.julia\packages\MLJBase\MuLnJ\src\machines.jl:464
┌ Info: Attempting to evaluate 10 models.
└ @ MLJTuning C:\Users\jeffr\.julia\packages\MLJTuning\Al9yX\src\tuned_models.jl:680
Evaluating over 10 metamodels: 100%[=========================] Time: 0:00:10


ProbabilisticPipeline(
    one_hot_encoder = OneHotEncoder(
            features = Symbol[],
            drop_last = false,
            ordered_factor = true,
            ignore = false),
    multinomial_classifier = MultinomialClassifier(
            lambda = 0.16681005372000587,
            gamma = 0.0,
            penalty = :l2,
            fit_intercept = true,
            penalize_intercept = false,
            scale_penalty_with_samples = true,
            solver = nothing),
    cache = true)

So it looks like it's useful to regularise a fair bit to get a lower cross entropy

In [28]:
ŷ = MLJ.predict(mtm, Xtrain)
cross_entropy(ŷ, ytrain) |> mean

0.6351726695229564

Interestingly this does not improve our missclassification rate

In [29]:
mcr= misclassification_rate(mode.(ŷ), ytrain)
println(rpad("MNC mcr: ", 10), round(mcr, sigdigits = 3))

MNC mcr:  0.278


We've probably reached the limit of a simple linear model.

### Trying another model

There are lots of categoricals, so maybe it's just better to use something that deals well with that like a tree-based classifier.

In [30]:
XGBC = @load XGBoostClassifier
dtc = machine(XGBC(), Xtrain, ytrain)
fit!(dtc)
ŷ = MLJ.predict(dtc, Xtrain)
cross_entropy(ŷ, ytrain) |> mean

import MLJXGBoostInterface

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\jeffr\.julia\packages\MLJModels\tMgLW\src\loading.jl:168


 ✔


┌ Warning: The scitype of `X`, in `machine(model, X, ...)` is incompatible with `model=XGBoostClassifier`:
│ scitype(X) = Table{Union{AbstractVector{Continuous}, AbstractVector{OrderedFactor{3}}, AbstractVector{OrderedFactor{4}}, AbstractVector{OrderedFactor{5}}, AbstractVector{OrderedFactor{2}}, AbstractVector{OrderedFactor{6}}}}
│ input_scitype(model) = Table{<:AbstractVector{<:Continuous}}.
└ @ MLJBase C:\Users\jeffr\.julia\packages\MLJBase\MuLnJ\src\machines.jl:133
┌ Info: Training Machine{XGBoostClassifier,…}.
└ @ MLJBase C:\Users\jeffr\.julia\packages\MLJBase\MuLnJ\src\machines.jl:464
[1]	train-mlogloss:0.844177
[2]	train-mlogloss:0.678176
[3]	train-mlogloss:0.564020
[4]	train-mlogloss:0.480493
[5]	train-mlogloss:0.416099
[6]	train-mlogloss:0.364831
[7]	train-mlogloss:0.329885
[8]	train-mlogloss:0.291539
[9]	train-mlogloss:0.263688
[10]	train-mlogloss:0.239011
[11]	train-mlogloss:0.218542
[12]	train-mlogloss:0.197875
[13]	train-mlogloss:0.181236
[14]	train-mlogloss:0.167084
[15]	

0.02560888f0

So we get a worse cross entropy but...



In [31]:
misclassification_rate(mode.(ŷ), ytrain)

0.0033444816053511705

a significantly better misclassification rate.

We could investigate more, do more tuning etc, but the key points of this tutorial was to show how to handle data with missing values.